In [36]:
!git log -1

commit 2213ea5854e8855191c347c0409500aba66d3173
Author: John McCallum <cfljam@users.noreply.github.com>
Date:   Tue Feb 23 14:52:01 2016 +1300

    updated notebook


In [37]:
!git remote -v

origin	https://github.com/cfljam/pcr_marker_design (fetch)
origin	https://github.com/cfljam/pcr_marker_design (push)
upstream	https://github.com/PlantandFoodResearch/pcr_marker_design (fetch)
upstream	https://github.com/PlantandFoodResearch/pcr_marker_design (push)


In [24]:
!pip freeze

anaconda-client==1.6.1
appnope==0.1.0
cffi==1.9.1
clyent==1.2.2
conda==4.2.13
cryptography==1.7.1
decorator==4.0.11
entrypoints==0.2.2
idna==2.2
ipykernel==4.5.2
ipython==5.2.2
ipython-genutils==0.1.0
ipywidgets==5.2.2
Jinja2==2.8
jsonschema==2.5.1
jupyter-client==5.0.0
jupyter-console==5.1.0
jupyter-core==4.3.0
MarkupSafe==0.23
mistune==0.7.3
nbconvert==4.2.0
nbformat==4.2.0
notebook==4.4.1
numpy==1.12.0
pandas==0.19.2
pexpect==4.2.1
pickleshare==0.7.3
prompt-toolkit==1.0.13
ptyprocess==0.5.1
pyasn1==0.1.9
pycosat==0.6.1
pycparser==2.17
pycrypto==2.6.1
Pygments==2.2.0
pyOpenSSL==16.2.0
python-dateutil==2.3
pytz==2016.10
PyYAML==3.12
pyzmq==16.0.2
qtconsole==4.2.1
requests==2.12.4
ruamel-yaml===-VERSION
simplegeneric==0.8.1
six==1.10.0
terminado==0.6
tornado==4.4.2
traitlets==4.3.1
wcwidth==0.1.7
widgetsnbextension==1.2.6
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
pwd

'/Users/johnmccallum/Documents/pcr_marker_design/docs'

### Load up bedtools 

In [34]:

from pybedtools import BedTool
from pyfaidx import Fasta

## Base this on the galaxy  test data

Note that this is wrong!

1. indel lengths arent correct
2. should be 1-based and start==stop for SNPs

need to revisit vcf2gvf

In [85]:
!head  -n 4 ../test/test-data/targets.gff

k69_93535	SAMTOOLS	INDEL	3	7	4.44	.	.	ID=k69_93535:SAMTOOLS:INDEL:3;Variant_seq=ATTTCACATCCCATCATCA,ATCACATCCCATCATCA;Reference_seq=ATCA;INDEL;DP=5;VDB=0.0149;AF1=1;AC1=24;DP4=0,0,3,0;MQ=23;FQ=-33.2
k69_93535	SAMTOOLS	INDEL	5	6	4.91	.	.	ID=k69_93535:SAMTOOLS:INDEL:5;Variant_seq=CTCATATCCCATCATC,CCGGCC;Reference_seq=C;INDEL;DP=6;VDB=0.0137;AF1=1;AC1=24;DP4=0,0,3,1;MQ=17;FQ=-32.4
k69_93535	SAMTOOLS	SNP	1141	1142	999	.	.	ID=k69_93535:SAMTOOLS:SNP:1141;Variant_seq=T;Reference_seq=C;DP=2644;VDB=0.0374;AF1=0.1882;AC1=5;DP4=748,786,225,294;MQ=42;FQ=999;PV4=0.037,0,0.036,0.39
k69_93535	SAMTOOLS	SNP	1147	1148	999	.	.	ID=k69_93535:SAMTOOLS:SNP:1147;Variant_seq=G;Reference_seq=C;DP=2645;VDB=0.0371;AF1=0.3527;G3=0.2771,0.7229,6.934e-153;HWE=0.0248;AC1=8;DP4=733,804,447,519;MQ=42;FQ=999;PV4=0.51,0,0.027,1


### Create a Bed Target Set for SNPs only

In [99]:
!awk -v OFS='\t' '/SNP/ {print $1,$4-1,$4}' ../test/test-data/targets.gff | tee  ../test/test-data/targets.snps.bed

k69_93535	1140	1141
k69_93535	1146	1147
k69_93535	1335	1336
k69_93535	2	3
k69_93535	660	661
k69_93535	750	751
k69_93535	886	887
k69_98089	1	2
k69_98089	29	30
k69_98089	2	3
k69_98089	452	453
k69_98089	514	515
k69_98089	521	522
k69_98089	549	550
k69_98089	624	625
k69_98089	692	693
k69_98089	716	717
k69_98089	722	723
k69_98089	725	726


Use the default Globals settings for Primer3

In [4]:
p3_globals={
        'PRIMER_OPT_SIZE': 20,
        'PRIMER_PICK_INTERNAL_OLIGO': 0,
        'PRIMER_INTERNAL_MAX_SELF_END': 8,
        'PRIMER_MIN_SIZE': 18,
        'PRIMER_MAX_SIZE': 25,
        'PRIMER_OPT_TM': 60.0,
        'PRIMER_MIN_TM': 57.0,
        'PRIMER_MAX_TM': 63.0,
        'PRIMER_MIN_GC': 20.0,
        'PRIMER_MAX_GC': 80.0,
        'PRIMER_MAX_POLY_X': 100,
        'PRIMER_INTERNAL_MAX_POLY_X': 100,
        'PRIMER_SALT_MONOVALENT': 50.0,
        'PRIMER_DNA_CONC': 50.0,
        'PRIMER_MAX_NS_ACCEPTED': 0,
        'PRIMER_MAX_SELF_ANY': 12,
        'PRIMER_MAX_SELF_END': 8,
        'PRIMER_PAIR_MAX_COMPL_ANY': 12,
        'PRIMER_PAIR_MAX_COMPL_END': 8,
        'PRIMER_PRODUCT_SIZE_RANGE': [60,250],
    }

### Load up the Module and Create a Designer Object

In [155]:
from pcr_marker_design import design as d
test_seq='../test/test-data/targets.fasta'
annfile='../test/test-data/targets.snps.bed'
des_test=d.PrimerDesign(test_seq,annfile,'Designer Test')

### Make a BedTool target

In [156]:
testBed=BedTool('../test/test-data/targets.gff')

In [157]:
[[X.length,X.start,X.end] for X in des_test.annotations]

[[1, 1140, 1141],
 [1, 1146, 1147],
 [1, 1335, 1336],
 [1, 2, 3],
 [1, 660, 661],
 [1, 750, 751],
 [1, 886, 887],
 [1, 1, 2],
 [1, 29, 30],
 [1, 2, 3],
 [1, 452, 453],
 [1, 514, 515],
 [1, 521, 522],
 [1, 549, 550],
 [1, 624, 625],
 [1, 692, 693],
 [1, 716, 717],
 [1, 722, 723],
 [1, 725, 726]]

### Create a Target

- base this on the test data targets file
- do this manually with the from_string option
- debtools is zero-indexed

In [158]:
target=BedTool("k69_93535 1146 1147",from_string=True)
print(target)

k69_93535	1146	1147



In [146]:
target[0].start

1146

In [131]:
target[0].end

1147

In [132]:
p3_globals['PRIMER_PRODUCT_SIZE_RANGE'][1]

250

In [160]:
seq_test=des_test.getseqslicedict(target,p3_globals['PRIMER_PRODUCT_SIZE_RANGE'][1])
seq_test

{'SEQUENCE_EXCLUDED_REGION': [(244, 1), (439, 1)],
 'SEQUENCE_ID': 'Designer Test',
 'SEQUENCE_TARGET': (250, 1),
 'SEQUENCE_TEMPLATE': 'AAATAATGGAGAATAGATGGTTCAAGAATGGATTCGAGCCTGTGAAATATTACATTGAGAATGATAGGTTTCATAAGTGGTGTAGCTTAGACGAAGAGAATGCTAATGACAACGAGGAGGTAGAATCTGGAGATGAATCAGACTCTTCAGTTGCTTCCTGCCCTCCTACACTTAATGAAGGAAAGAAAAAAAGGACAGGGAAGCTTCATAGGCCTTTGAGTCTGAACGCATTTGACATAATTTCCTTTTCCAGAGGATTTGATCTTTCAGGTTTGTTTGAAGAAACGGGAGATGAAACAAGATTTGTGTCGGGTGAAACGATACCAAACATCATATCGAAATTGGAGGAGATTGCAAAAGTGGGTAGTTTCACGTTTAGGAAGAAGGATTGTAGGGTTAGTTTAGAAGGAACGCGAGAAGGAGTGAAGGGCCCTCTTACGATTGGAGCTGAGATATTTGAGCTTACGCCTAGTTTGGTTGTTGTTGAGCTTAAGAAGAAAG',
 'TARGET_ID': 'k69_93535_1146_1147'}

In [161]:
str(seq_test['TARGET_ID'])

'k69_93535_1146_1147'

In [162]:
import primer3 as P3

### Design Using the Slice and P3 py directly

In [170]:
P3.designPrimers(seq_args=seq_test,
                 global_args= p3_globals)

{'PRIMER_INTERNAL_EXPLAIN': 'considered 24, unacceptable product size 19, ok 5',
 'PRIMER_INTERNAL_NUM_RETURNED': 0,
 'PRIMER_LEFT_0': (194, 20),
 'PRIMER_LEFT_0_END_STABILITY': 5.19,
 'PRIMER_LEFT_0_GC_PERCENT': 55.0,
 'PRIMER_LEFT_0_HAIRPIN_TH': 0.0,
 'PRIMER_LEFT_0_PENALTY': 0.5485098850288068,
 'PRIMER_LEFT_0_SELF_ANY_TH': 27.01947809412121,
 'PRIMER_LEFT_0_SELF_END_TH': 18.001360791532136,
 'PRIMER_LEFT_0_SEQUENCE': 'ACAGGGAAGCTTCATAGGCC',
 'PRIMER_LEFT_0_TM': 59.45149011497119,
 'PRIMER_LEFT_1': (194, 20),
 'PRIMER_LEFT_1_END_STABILITY': 5.19,
 'PRIMER_LEFT_1_GC_PERCENT': 55.0,
 'PRIMER_LEFT_1_HAIRPIN_TH': 0.0,
 'PRIMER_LEFT_1_PENALTY': 0.5485098850288068,
 'PRIMER_LEFT_1_SELF_ANY_TH': 27.01947809412121,
 'PRIMER_LEFT_1_SELF_END_TH': 18.001360791532136,
 'PRIMER_LEFT_1_SEQUENCE': 'ACAGGGAAGCTTCATAGGCC',
 'PRIMER_LEFT_1_TM': 59.45149011497119,
 'PRIMER_LEFT_2': (194, 20),
 'PRIMER_LEFT_2_END_STABILITY': 5.19,
 'PRIMER_LEFT_2_GC_PERCENT': 55.0,
 'PRIMER_LEFT_2_HAIRPIN_TH': 0.0,
 'P

### Design Using Module

In [172]:
from pcr_marker_design import run_p3

In [173]:
run_p3.run_P3(global_dict=p3_globals,target_dict=seq_test)

[{'PRIMER_LEFT': (194, 20),
  'PRIMER_LEFT_SEQUENCE': 'ACAGGGAAGCTTCATAGGCC',
  'PRIMER_RIGHT': (424, 20),
  'PRIMER_RIGHT_SEQUENCE': 'CACTCCTTCTCGCGTTCCTT'},
 {'PRIMER_LEFT': (194, 20),
  'PRIMER_LEFT_SEQUENCE': 'ACAGGGAAGCTTCATAGGCC',
  'PRIMER_RIGHT': (428, 20),
  'PRIMER_RIGHT_SEQUENCE': 'CCTTCACTCCTTCTCGCGTT'},
 {'PRIMER_LEFT': (194, 20),
  'PRIMER_LEFT_SEQUENCE': 'ACAGGGAAGCTTCATAGGCC',
  'PRIMER_RIGHT': (426, 20),
  'PRIMER_RIGHT_SEQUENCE': 'TTCACTCCTTCTCGCGTTCC'},
 {'PRIMER_LEFT': (209, 20),
  'PRIMER_LEFT_SEQUENCE': 'AGGCCTTTGAGTCTGAACGC',
  'PRIMER_RIGHT': (424, 20),
  'PRIMER_RIGHT_SEQUENCE': 'CACTCCTTCTCGCGTTCCTT'}]

### Try Design around a longer Target INterval

use this magic to force reload when editing source

In [147]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [154]:
from pcr_marker_design import design as d
test_seq="../test/test-data/targets.fasta"
annfile= "../test/test-data/targets.bed"
k69_93535_250_290='GACAAAGAGAAAATCCTCAAATCCGGCCTCGTCAACCACA'
designer = d.PrimerDesign(test_seq,annfile,'Test')
seqslice=designer.getseqslicedict(target,250)

ValueError: File "../test/test-data/targets.bed" does not exist

In [153]:
[[X.length,X.start,X.end] for X in designer.annotations]

[[5, 2, 7],
 [2, 4, 6],
 [2, 1140, 1142],
 [2, 1146, 1148],
 [2, 1335, 1337],
 [2, 2, 4],
 [2, 660, 662],
 [2, 750, 752],
 [2, 886, 888],
 [5, 1, 6],
 [4, 2, 6],
 [3, 3, 6],
 [2, 4, 6],
 [2, 721, 723],
 [2, 725, 727],
 [3, 728, 731],
 [8, 740, 748],
 [7, 741, 748],
 [6, 742, 748],
 [2, 1, 3],
 [2, 29, 31],
 [2, 2, 4],
 [2, 452, 454],
 [2, 514, 516],
 [2, 521, 523],
 [2, 549, 551],
 [2, 624, 626],
 [2, 692, 694],
 [2, 716, 718],
 [2, 722, 724],
 [2, 725, 727]]

In [151]:
seqslice

{'SEQUENCE_EXCLUDED_REGION': [(244, 2), (439, 2)],
 'SEQUENCE_ID': 'Test',
 'SEQUENCE_TARGET': (250, 1),
 'SEQUENCE_TEMPLATE': 'AAATAATGGAGAATAGATGGTTCAAGAATGGATTCGAGCCTGTGAAATATTACATTGAGAATGATAGGTTTCATAAGTGGTGTAGCTTAGACGAAGAGAATGCTAATGACAACGAGGAGGTAGAATCTGGAGATGAATCAGACTCTTCAGTTGCTTCCTGCCCTCCTACACTTAATGAAGGAAAGAAAAAAAGGACAGGGAAGCTTCATAGGCCTTTGAGTCTGAACGCATTTGACATAATTTCCTTTTCCAGAGGATTTGATCTTTCAGGTTTGTTTGAAGAAACGGGAGATGAAACAAGATTTGTGTCGGGTGAAACGATACCAAACATCATATCGAAATTGGAGGAGATTGCAAAAGTGGGTAGTTTCACGTTTAGGAAGAAGGATTGTAGGGTTAGTTTAGAAGGAACGCGAGAAGGAGTGAAGGGCCCTCTTACGATTGGAGCTGAGATATTTGAGCTTACGCCTAGTTTGGTTGTTGTTGAGCTTAAGAAGAAAG',
 'TARGET_ID': 'k69_93535_1146_XX1147'}

In [145]:
[X for X in designer.annotations if X.chrom=="k69_93535"]

[Interval(k69_93535:2-7),
 Interval(k69_93535:4-6),
 Interval(k69_93535:1140-1142),
 Interval(k69_93535:1146-1148),
 Interval(k69_93535:1335-1337),
 Interval(k69_93535:2-4),
 Interval(k69_93535:660-662),
 Interval(k69_93535:750-752),
 Interval(k69_93535:886-888)]

In [81]:
[X for X in designer.annotations if (X.chrom=="k69_93535") & (X.start >1000)& (X.end < 1500)]

[Interval(k69_93535:1140-1142),
 Interval(k69_93535:1146-1148),
 Interval(k69_93535:1335-1337)]

In [60]:
foo['zip'=1234]

SyntaxError: invalid syntax (<ipython-input-60-584824af9423>, line 1)

In [55]:
seqslice

u'GACAAAGAGAAAATCCTCAAATCCGGCCTCGTCAACCACA'

In [14]:
[X  for X in designer.annotations]

[Interval(k69_93535:2-7),
 Interval(k69_93535:4-6),
 Interval(k69_93535:1140-1142),
 Interval(k69_93535:1146-1148),
 Interval(k69_93535:1335-1337),
 Interval(k69_93535:2-4),
 Interval(k69_93535:660-662),
 Interval(k69_93535:750-752),
 Interval(k69_93535:886-888),
 Interval(k69_98089:1-6),
 Interval(k69_98089:2-6),
 Interval(k69_98089:3-6),
 Interval(k69_98089:4-6),
 Interval(k69_98089:721-723),
 Interval(k69_98089:725-727),
 Interval(k69_98089:728-731),
 Interval(k69_98089:740-748),
 Interval(k69_98089:741-748),
 Interval(k69_98089:742-748),
 Interval(k69_98089:1-3),
 Interval(k69_98089:29-31),
 Interval(k69_98089:2-4),
 Interval(k69_98089:452-454),
 Interval(k69_98089:514-516),
 Interval(k69_98089:521-523),
 Interval(k69_98089:549-551),
 Interval(k69_98089:624-626),
 Interval(k69_98089:692-694),
 Interval(k69_98089:716-718),
 Interval(k69_98089:722-724),
 Interval(k69_98089:725-727)]

In [17]:
designer.annotations[3].attrs

{'AC1': '8',
 'AF1': '0.3527',
 'DP': '2645',
 'DP4': '733,804,447,519',
 'FQ': '999',
 'G3': '0.2771,0.7229,6.934e-153',
 'HWE': '0.0248',
 'ID': 'k69_93535:SAMTOOLS:SNP:1147',
 'MQ': '42',
 'PV4': '0.51,0,0.027,1',
 'Reference_seq': 'C',
 'VDB': '0.0371',
 'Variant_seq': 'G'}

In [20]:
b=BedTool("k69_93535 1000 1500",from_string=True)

In [22]:
print(b.intersect(designer.annotations))

k69_93535	1141	1142
k69_93535	1147	1148
k69_93535	1336	1337



In [26]:
vcf_test= '/Users/cfljam/Documents/galaxy-pcr-markers/test-data/freebayes_out.vcf'

In [29]:
c=BedTool(vcf_test)
d=BedTool("Chr1 1000 1500",from_string=True)

In [34]:
[(x. start,x.length) for x in c.intersect(d)[10:120]]

[(1460, 1), (1480, 1)]

In [30]:
print(c.intersect(d))

Chr1	1007	.	T	A	17.1701	.	AB=0;ABP=0;AC=2;AF=1;AN=2;AO=1;CIGAR=1X;DP=1;DPRA=0;EPP=5.18177;EPPR=0;HWE=-0;LEN=1;MEANALT=1;MQM=40;MQMR=0;NS=1;NUMALT=1;ODDS=3.91202;PAIRED=0;PAIREDR=0;RO=0;RPP=5.18177;RPPR=0;RUN=1;SAP=5.18177;SRP=0;TYPE=snp;XAI=0.0103093;XAM=0.0309278;XAS=0.0206186;XRI=0;XRM=0;XRS=0;BVAR	GT:GQ:DP:RO:QR:AO:QA:GL	1/1:14.1547:1:0:0:1:17:-1.7,-0.30103,0
Chr1	1012	.	TGT	AGA	17.1701	.	AB=0;ABP=0;AC=2;AF=1;AN=2;AO=1;CIGAR=1X1M1X;DP=1;DPRA=0;EPP=5.18177;EPPR=0;HWE=-0;LEN=3;MEANALT=1;MQM=40;MQMR=0;NS=1;NUMALT=1;ODDS=3.91202;PAIRED=0;PAIREDR=0;RO=0;RPP=5.18177;RPPR=0;RUN=1;SAP=5.18177;SRP=0;TYPE=complex;XAI=0;XAM=0.0309278;XAS=0.0309278;XRI=0;XRM=0;XRS=0;BVAR	GT:GQ:DP:RO:QR:AO:QA:GL	1/1:14.1547:1:0:0:1:17:-1.7,-0.30103,0
Chr1	1056	.	A	C	17.1701	.	AB=0;ABP=0;AC=2;AF=1;AN=2;AO=1;CIGAR=1X;DP=1;DPRA=0;EPP=5.18177;EPPR=0;HWE=-0;LEN=1;MEANALT=1;MQM=40;MQMR=0;NS=1;NUMALT=1;ODDS=3.91202;PAIRED=0;PAIREDR=0;RO=0;RPP=5.18177;RPPR=0;RUN=1;SAP=5.18177;SRP=0;TYPE=snp;XAI=0.0103093;XAM=0.0309278;X